In [4]:
# 3 : 24

#Importing libraries

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Reading the data

In [6]:
df = pd.read_csv('/content/diabetic_data.csv')

#Exploring the data

shape

In [7]:
df.shape

(101766, 50)

info and description

In [8]:
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [9]:
# df.info()

head and tail

In [10]:
# df.head()

In [11]:
# df.tail()

In [12]:
df.isnull().sum()
print()

In [13]:
# There are some unnecessary columns, and as we have huge amount of columns we
# can drop the "not very useful" ones
df.drop(columns = ["encounter_id", "patient_nbr"], inplace = True)
print()
# for now this is enough, later when we underestand better which features do no
# goof to our model we will delete mroe

#Data cleaning

#Dealing with nulls and missing values

In [14]:
# Because a huge amount of data is missing from max_glu_serum and A1Cresult and
# it is not possible to properly restore we will drop them
df.drop(columns = ["max_glu_serum", "A1Cresult"], inplace = True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 46 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      101766 non-null  object
 1   gender                    101766 non-null  object
 2   age                       101766 non-null  object
 3   weight                    101766 non-null  object
 4   admission_type_id         101766 non-null  int64 
 5   discharge_disposition_id  101766 non-null  int64 
 6   admission_source_id       101766 non-null  int64 
 7   time_in_hospital          101766 non-null  int64 
 8   payer_code                101766 non-null  object
 9   medical_specialty         101766 non-null  object
 10  num_lab_procedures        101766 non-null  int64 
 11  num_procedures            101766 non-null  int64 
 12  num_medications           101766 non-null  int64 
 13  number_outpatient         101766 non-null  int64 
 14  numb

#Analyzing all features

race

In [16]:
df['race'].unique()
# we see that there is both "?" and "Other" and it would not be a big problem to
# combine them
df['race'] = df['race'].replace("?", "Other")

gender

In [17]:
df['gender'].unique()
# We will lwave like this for a while

array(['Female', 'Male', 'Unknown/Invalid'], dtype=object)

age

In [18]:
df['age'].unique()

array(['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)',
       '[60-70)', '[70-80)', '[80-90)', '[90-100)'], dtype=object)

weight

**USING KNN TO FILL MISSING VALUES**

In [19]:
from sklearn.impute import KNNImputer

weight_map = {
    '[0-25)': 12.5,
    '[25-50)': 37.5,
    '[50-75)': 62.5,
    '[75-100)': 87.5,
    '[100-125)': 112.5,
    '[125-150)': 137.5,
    '[150-175)': 162.5,
    '[175-200)': 187.5,
    '>200': 210,
    '?': np.nan
}
df['weight_num'] = df['weight'].replace(weight_map).astype(float)

numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

imputer = KNNImputer(n_neighbors=3)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

def num_to_range(x):
    if x < 25: return '[0-25)'
    elif x < 50: return '[25-50)'
    elif x < 75: return '[50-75)'
    elif x < 100: return '[75-100)'
    elif x < 125: return '[100-125)'
    elif x < 150: return '[125-150)'
    elif x < 175: return '[150-175)'
    elif x < 200: return '[175-200)'
    else: return '>200'

df['weight'] = df['weight_num'].apply(num_to_range)

df.drop(columns=['weight_num'], inplace=True)

print(df['weight'].unique())
print(df.head())


/tmp/ipython-input-507576756.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['weight_num'] = df['weight'].replace(weight_map).astype(float)


['[50-75)' '[75-100)' '[100-125)' '[25-50)' '[125-150)' '[175-200)'
 '[0-25)' '[150-175)' '>200']
              race  gender      age     weight  admission_type_id  \
0        Caucasian  Female   [0-10)    [50-75)                6.0   
1        Caucasian  Female  [10-20)   [75-100)                1.0   
2  AfricanAmerican  Female  [20-30)   [75-100)                1.0   
3        Caucasian    Male  [30-40)    [50-75)                1.0   
4        Caucasian    Male  [40-50)  [100-125)                1.0   

   discharge_disposition_id  admission_source_id  time_in_hospital payer_code  \
0                      25.0                  1.0               1.0          ?   
1                       1.0                  7.0               3.0          ?   
2                       1.0                  7.0               2.0          ?   
3                       1.0                  7.0               2.0          ?   
4                       1.0                  7.0               1.0          ?   



the rest (lazy to write down all)

In [20]:
df['admission_type_id'].unique()

array([6., 1., 2., 3., 4., 5., 8., 7.])

In [53]:
df['discharge_disposition_id'].unique()
df['admission_source_id'].unique()
df['time_in_hospital'].unique()
df['payer_code'].unique()
payer_mode = df['payer_code'].mode()[0]
df['payer_code'] = df['payer_code'].replace("?", payer_mode)

In [55]:
df['medical_specialty'].unique()
medical_mode = df['medical_specialty'].mode()[0]
df['medical_specialty'] = df['medical_specialty'].replace("?", medical_mode)

In [56]:
# As from medical speciality to diag_1 everything is int64 no need to check
# Whats wrong because nothing is

In [62]:
original_df = pd.read_csv('/content/diabetic_data.csv')

df['diag_1'] = original_df['diag_1']

print(df['diag_1'].isnull().sum())

0


In [63]:
# df['diag_1'].unique()

In [66]:
def is_numeric(value):
    if value is None:
        return False
    try:
        float(value)
        return True
    except ValueError:
        return False

non_numeric_diag1_values = [value for value in df['diag_1'].unique() if not is_numeric(value)]
# print("Non-numeric values in 'diag_1':")
# for value in non_numeric_diag1_values:
    # print(value)

In [69]:
df['diag_1'].unique()
mode_diag1 = df['diag_1'].mode()[0]
df['diag_1'] = df['diag_1'].replace("?", mode_diag1)

df['diag_2'].unique()
mode_diag2 = df['diag_2'].mode()[0]
df['diag_2'] = df['diag_2'].replace("?", mode_diag2)

df['diag_3'].unique()
mode_diag3 = df['diag_3'].mode()[0]
df['diag_3'] = df['diag_3'].replace("?", mode_diag2)

#MAGIC **CODE**

In [72]:
import numpy as np

missing_values = ["?", "Unknown", "invalid", np.nan, None]

for col in df.select_dtypes(include='object').columns:
   if df[col].isin(missing_values).any():
        mode_value = df[col][~df[col].isin(missing_values)].mode()
        if not mode_value.empty:
            df[col] = df[col].replace(missing_values, mode_value[0])
            print(f"Replaced missing values in '{col}' with mode: {mode_value[0]}")
        else:
            print(f"Could not calculate mode for '{col}' (column contains only missing values or is empty after excluding missing values).")

print("\nMissing values after replacement:")
for col in df.columns:
    if df[col].isin(missing_values).any().any():
        print(f"Column '{col}' still contains missing values.")


Missing values after replacement:


In [ ]:
df.drop(columns = ["readmitted_NO"], inplace = True)

#Encoding

In [81]:
categorical_cols = df.select_dtypes(include=['object']).columns

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Convert boolean columns to integers (0s and 1s)
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'bool':
        df_encoded[col] = df_encoded[col].astype(int)

display(df_encoded.head())

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes,readmitted_>30,readmitted_NO
0,6.0,25.0,1.0,1.0,41.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
1,1.0,1.0,7.0,3.0,59.0,0.0,18.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,1,0
2,1.0,1.0,7.0,2.0,11.0,5.0,13.0,2.0,0.0,1.0,...,0,0,0,0,0,0,1,1,0,1
3,1.0,1.0,7.0,2.0,44.0,1.0,16.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,1
4,1.0,1.0,7.0,1.0,51.0,0.0,8.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,1


#Feature creation

In [82]:
df['total_visits'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
display(df[['number_outpatient', 'number_emergency', 'number_inpatient', 'total_visits']].head())

,number_outpatient,number_emergency,number_inpatient,total_visits
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,2.0,0.0,1.0,3.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


#Train/test

In [83]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop(['readmitted_>30', 'readmitted_NO'], axis=1)
y = df_encoded['readmitted_>30'] # Using 'readmitted_>30' as the target for binary classification

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Scaling

In [84]:
# almost always scaling does no bad, so let us scale
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Model creation

#LINEAR REGRESSION

In [85]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [86]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# accuracy 0.65 is not bad for a simple linear model but is not good either to
# leave it like that
# precision and recall have very bad resulets, do does f1 score

Accuracy: 0.6530
Precision: 0.5124
Recall: 0.1592
F1-score: 0.2429


#DECISION TREES

In [88]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_scaled, y_train)

y_pred_dt = dt_model.predict(X_test_scaled)

accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

print(f"Decision Tree Accuracy: {accuracy_dt:.4f}")
print(f"Decision Tree Precision: {precision_dt:.4f}")
print(f"Decision Tree Recall: {recall_dt:.4f}")
print(f"Decision Tree F1-score: {f1_dt:.4f}")

Decision Tree Accuracy: 0.5908
Decision Tree Precision: 0.4100
Decision Tree Recall: 0.3878
Decision Tree F1-score: 0.3986


#Comparing two models

In [89]:
print("--- Model Comparison ---")
print("Metric          | Logistic Regression | Decision Tree")
print("----------------|---------------------|---------------")
print(f"Accuracy        | {accuracy:.4f}            | {accuracy_dt:.4f}")
print(f"Precision       | {precision:.4f}            | {precision_dt:.4f}")
print(f"Recall          | {recall:.4f}            | {recall_dt:.4f}")
print(f"F1-score        | {f1:.4f}            | {f1_dt:.4f}")

--- Model Comparison ---
Metric          | Logistic Regression | Decision Tree
----------------|---------------------|---------------
Accuracy        | 0.6530            | 0.5908
Precision       | 0.5124            | 0.4100
Recall          | 0.1592            | 0.3878
F1-score        | 0.2429            | 0.3986


#Analyzing comparison, analyze in general

In [ ]:
# for our model it is better to have fake positivs than fake negatives because
# - if we be more attantive with healthy poeple(fake positive) nothing bad will happen
#   except for lost time, but it will raise the hospital's rating

# - if we be less attantive with sick people( fake negative) they might die. Lost life,
#   bad rating, saved resources. In this case the resourses ait important enough


# It is very important for us to focus on recall(I filled missing values incorreclty and didnt focus on it)
# both models aint good but if we choose one, decision trees is better

# Ensemble learning

In [90]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

y_pred_rf = rf_model.predict(X_test_scaled)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Ensemble Model Accuracy: {accuracy_rf:.4f}")
print(f"Ensemble Model Precision: {precision_rf:.4f}")
print(f"Ensemble Model Recall: {recall_rf:.4f}")
# unfortunately the recall is not good enough because of the way i filled our
# missing values,, should have focused more on recall and less or precision
print(f"Ensemble Model F1-score: {f1_rf:.4f}")

# randomforest aint good either. nothing can be done except for fixing the data given to the model.

Ensemble Model Accuracy: 0.6582
Ensemble Model Precision: 0.5388
Ensemble Model Recall: 0.1550
Ensemble Model F1-score: 0.2407


## Identify the best model

### Subtask:
Determine the best performing model from the initial comparison based on F1-score.


In [91]:
print("--- Model Comparison ---")
print("Metric          | Logistic Regression | Decision Tree | Random Forest")
print("----------------|---------------------|---------------|---------------")
print(f"Accuracy        | {accuracy:.4f}            | {accuracy_dt:.4f}            | {accuracy_rf:.4f}")
print(f"Precision       | {precision:.4f}            | {precision_dt:.4f}            | {precision_rf:.4f}")
print(f"Recall          | {recall:.4f}            | {recall_dt:.4f}            | {recall_rf:.4f}")
print(f"F1-score        | {f1:.4f}            | {f1_dt:.4f}            | {f1_rf:.4f}")

# Determine the best model based on F1-score
f1_scores = {
    'Logistic Regression': f1,
    'Decision Tree': f1_dt,
    'Random Forest': f1_rf
}

best_model_f1 = max(f1_scores, key=f1_scores.get)
print(f"\nBased on F1-score, the best performing model is: {best_model_f1}")

# decision tree is!

--- Model Comparison ---
Metric          | Logistic Regression | Decision Tree | Random Forest
----------------|---------------------|---------------|---------------
Accuracy        | 0.6530            | 0.5908            | 0.6582
Precision       | 0.5124            | 0.4100            | 0.5388
Recall          | 0.1592            | 0.3878            | 0.1550
F1-score        | 0.2429            | 0.3986            | 0.2407

Based on F1-score, the best performing model is: Decision Tree
